In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import os
import time
import boto3
import botocore
from pathlib import Path
from huggingface_hub import login

In [ ]:
#Some models require a hugginface login; uncomment the following code
#to enable an interactive login screen where you can provide your hugginface credentials
#from huggingface_hub import notebook_login.
#For some models, you may also need to login to Huggingface using your browser and agree to a use policy.
#notebook_login()

In [ ]:
#Setup variables for this script

In [ ]:
models_dir = Path.cwd() / "models"
models_dir.mkdir(parents=True, exist_ok=True)
model_prefix = "GritLM"
model_name = "GritLM-7B"
full_model_name = f"{model_prefix}/{model_name}"
save_path = models_dir / model_prefix

In [ ]:
#Download model

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id=full_model_name, local_dir=save_path)

In [ ]:
#Get the s3 connection parameters (these are entered in the workbench setup screens)

In [ ]:
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

In [ ]:
session = boto3.session.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
          )

In [ ]:
s3_resource = session.resource(
               's3',
               config=botocore.client.Config(signature_version='s3v4'),
               endpoint_url=endpoint_url,
               region_name=region_name
              )
bucket = s3_resource.Bucket(bucket_name)
                        
def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
                                
def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
#Upload the model to an S3 bucket within the cluster

In [ ]:
upload_directory_to_s3(save_path, model_name)